# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from statistics import mean

In [2]:
RANDOM_STATE = 12345
PRODUCT_PROFIT = 450000
BUDGET = 10000000000
HOLES_TARGET_COUNT = 200
ALPHA = 0.025

In [3]:
df_0 = pd.read_csv('datasets/geo_data_0.csv')
print(df_0.info())
df_0.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
df_1 = pd.read_csv('datasets/geo_data_1.csv')
print(df_1.info())
df_1.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
df_2 = pd.read_csv('datasets/geo_data_2.csv')
print(df_2.info())
df_2.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


1. Пропусков в данных нет. Оставляем как есть.
2. Имеются отрицательные значения, которые, возможно, связаны с качеством скважины. Оставляем как есть.

## Обучение и проверка модели

In [6]:
df = {0: df_0, 1: df_1, 2: df_2}

In [7]:
predictions = {}
targets = {}
rmse_scores = {}

for key, i in df.items():
    features = i.drop(['id', 'product'], axis=1)
    target = i['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=RANDOM_STATE)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predictions_valid) ** 0.5
    rmse_scores[key] = rmse
    predictions[key] = pd.Series(predictions_valid)
    targets[key] = target_valid.reset_index(drop=True)
    

In [8]:
def regions_info(key):
    region_predictions = predictions[key]
    region_predictions_mean = round(region_predictions.mean(), 2)
    region_rmse = rmse_scores[key]
    
    return region_predictions, region_predictions_mean, region_rmse

In [9]:
predictions_0, predictions_0_mean, rmse_0 = regions_info(0)

print('Средний объём запасов в скважине (тыс. баррелей):', predictions_0_mean)
print('RMSE модели:', rmse_0)

Средний объём запасов в скважине (тыс. баррелей): 92.59
RMSE модели: 37.5794217150813


In [10]:
predictions_1, predictions_1_mean, rmse_1 = regions_info(1)

print('Средний объём запасов в скважине (тыс. баррелей):', predictions_1_mean)
print('RMSE модели:', rmse_1)

Средний объём запасов в скважине (тыс. баррелей): 68.73
RMSE модели: 0.8930992867756182


In [11]:
predictions_2, predictions_2_mean, rmse_2 = regions_info(2)

print('Средний объём запасов в скважине (тыс. баррелей):', predictions_2_mean)
print('RMSE модели:', rmse_2)

Средний объём запасов в скважине (тыс. баррелей): 94.97
RMSE модели: 40.02970873393434


Средний объём запасов в скважине больше в регионе 2, затем 0. Но качество модели лучше в регионе 1.

## Подготовка к расчёту прибыли

In [12]:
loss_free_volume = BUDGET / HOLES_TARGET_COUNT / PRODUCT_PROFIT
print(f'Достаточный объём сырья для безубыточной разработки новой скважины: {round(loss_free_volume, 2)} единиц продукта')


Достаточный объём сырья для безубыточной разработки новой скважины: 111.11 единиц продукта


Достаточный объём сырья для безубыточной разработки новой скважины больше, чем средний объем запасов скважин в каждом из трех регионов. При этом по средему объёму запасов в скважине сильно отстает регион 1.


## Расчёт прибыли и рисков 

In [13]:
def best_holes_profit(pred, targ):
    best_predictions = pred.sort_values(ascending=False).head(200)
    best_target = targ.loc[best_predictions.index].head(200)
    
    common_volume = best_target.sum()
    profit = (common_volume * PRODUCT_PROFIT) - BUDGET
    return profit

In [14]:
profit_0 = best_holes_profit(predictions[0], targets[0])
print('Прибыль от 200 лучших скважин 0 региона:', profit_0)

Прибыль от 200 лучших скважин 0 региона: 3320826043.1398506


In [15]:
profit_1 = best_holes_profit(predictions[1], targets[1])
print('Прибыль от 200 лучших скважин региона 1:', profit_1)

Прибыль от 200 лучших скважин региона 1: 2415086696.681511


In [16]:
profit_2 = best_holes_profit(predictions[2], targets[2])
print('Прибыль от 200 лучших скважин региона 2:', profit_2)

Прибыль от 200 лучших скважин региона 2: 2710349963.5998325


По результатам данных прибыль от 200 лучших скважин была больше в регионе 0, затем 2.

In [17]:
state = np.random.RandomState(RANDOM_STATE)

def profit_distribution(key):
    profits = []
    
    for i in range(1000):
        predictions_subsample = predictions[key].sample(n=500, replace=True, random_state=state)
        target_subsample = targets[key].loc[predictions_subsample.index]
        
        profit = best_holes_profit(predictions_subsample, target_subsample)
        profits.append(profit)
    
    profits_df = pd.Series(profits)
    profit_mean = profits_df.mean()
    risk_share = (profits_df < 0).mean()
    
    lower = profits_df.quantile(0.025)
    upper = profits_df.quantile(0.975)
    profits_interval = (lower, upper)

    return risk_share, profit_mean, profits_interval


In [18]:
risk_share_0, profit_mean_0, interval_0 = profit_distribution(0)

print(f'Вероятность убытков равна {risk_share_0:.2%}')
print(f'95%-й доверительный интервал: {interval_0}')
print(f'Средняя прибыль в регионе: {profit_mean_0}')

if risk_share_0 < ALPHA:
    print('Регион с низким уровнем риска - подходит по критериям')
else:
    print('Регион с высоким уровнем риска - не подходит по критериям')

Вероятность убытков равна 6.00%
95%-й доверительный интервал: (-102090094.83793654, 947976353.3583689)
Средняя прибыль в регионе: 425938526.910592
Регион с высоким уровнем риска - не подходит по критериям


In [19]:
risk_share_1, profit_mean_1, interval_1 = profit_distribution(1)
print(f'Вероятность убытков равна {risk_share_1:.2%}')
print(f'95%-й доверительный интервал: {interval_1}')
print(f'Средняя прибыль в регионе: {profit_mean_1}')

if risk_share_1 < ALPHA:
    print('Регион с низким уровнем риска - подходит по критериям')
else:
    print('Регион с высоким уровнем риска - не подходит по критериям')

Вероятность убытков равна 0.30%
95%-й доверительный интервал: (128123231.43308444, 953612982.0669085)
Средняя прибыль в регионе: 518259493.69732505
Регион с низким уровнем риска - подходит по критериям


In [20]:
risk_share_2, profit_mean_2, interval_2 = profit_distribution(2)
print(f'Вероятность убытков равна {risk_share_2:.2%}')
print(f'95%-й доверительный интервал: {interval_2}')
print(f'Средняя прибыль в регионе: {profit_mean_2}')

if risk_share_2 < ALPHA:
    print('Регион с низким уровнем риска - подходит по критериям')
else:
    print('Регион с высоким уровнем риска - не подходит по критериям')

Вероятность убытков равна 6.20%
95%-й доверительный интервал: (-115852609.16001143, 989629939.8445739)
Средняя прибыль в регионе: 420194005.3440499
Регион с высоким уровнем риска - не подходит по критериям


## Вывод

Безубыточным оказался только регион 1 - (вероятность убытков в нем меньше 2.5%). Среди них выбираем регион с наибольшей средней прибылью. Средняя прибыль от каждой скважины в регионе составляет 518259493 рублей.